In [1]:
!pip install llama-index langchain-community llama-index-llms-gemini

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from llama_index.core.tools import FunctionTool
from llama_index.llms.gemini import Gemini
from llama_index.core.agent import ReActAgent
from google.colab import userdata

# Set API Key
os.environ['GEMINI_API_KEY'] = userdata.get('gemini-key')
csv_path = '/content/combined_file.csv'  # Update with your CSV path
# Initialize LLM
llm = Gemini(api_key=os.getenv("GEMINI_API_KEY"), model="models/gemini-1.5-flash-latest")

<ipython-input-2-6f89dc5998f9>:13: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(api_key=os.getenv("GEMINI_API_KEY"), model="models/gemini-1.5-flash-latest")


In [3]:
# ----- Tool Definitions with Reasoning Prompts -----
def sum_column(column: str, csv_path=csv_path) -> str:
    """Calculate sum of a column. Use when user asks for total or summation."""
    df = pd.read_csv(csv_path)
    if column not in df.columns:
        return f"Error: Column '{column}' not found"
    return f"The sum of {column} is {df[column].sum()}"

def compare_columns(col1: str, col2: str, csv_path=csv_path) -> str:
    """Compare averages of two columns. Use for comparison requests."""
    df = pd.read_csv(csv_path)
    for col in [col1, col2]:
        if col not in df.columns:
            return f"Error: Column '{col}' not found"
    avg1, avg2 = df[col1].mean(), df[col2].mean()
    return f"Average: {col1}={avg1:.2f}, {col2}={avg2:.2f}. {'Higher' if avg1 > avg2 else 'Lower'} column: {col1 if avg1 > avg2 else col2}"

def plot_column(column: str, csv_path=csv_path) -> str:
    """Generate bar chart for a column. Use when visualization is requested."""
    df = pd.read_csv(csv_path)
    if column not in df.columns:
        return f"Error: Column '{column}' not found"
    plt.figure(figsize=(10,4))
    df[column].plot(kind='bar')
    plt.title(f"Distribution of {column}")
    plt.savefig('plot.png')
    plt.close()
    return "Chart saved to plot.png"

# Create tools
tools = [
    FunctionTool.from_defaults(fn=sum_column, name="sum_column"),
    FunctionTool.from_defaults(fn=compare_columns, name="compare_columns"),
    FunctionTool.from_defaults(fn=plot_column, name="plot_column")
]


In [4]:
# ----- Create Agent with Reasoning Capabilities -----
class ReasoningAgent(ReActAgent):
    def _process_response(self, response):
        """Add reasoning visualization"""
        print(f"\n🤔 Agent Thoughts:")
        for step in response.response_gen:
            if step.action_reason:
                print(f"  - {step.action_reason}")
            if step.observation:
                print(f"  ✅ Observed: {step.observation}")
        return super()._process_response(response)

# Initialize agent
agent = ReasoningAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True,
    system_prompt="""You are an analytical assistant. Follow these steps:
    1. Understand the request in Vietnamese/English
    2. Check data availability
    3. Choose appropriate tool
    4. Verify results
    Respond in the user's language."""
)

# ----- Run Interaction Loop -----
print("📊 CSV Analysis Agent (Type 'exit' to quit)")
while True:
    query = input("\nYou: ")
    if query.lower() == 'exit':
        break

    print("\n🔍 Analyzing...")
    response = agent.chat(query)

    print(f"\n📝 Final Answer:")
    print(str(response))
    print("\n" + "-"*50)

print("Session ended. Goodbye!")

📊 CSV Analysis Agent (Type 'exit' to quit)

You: where is viet nam

🔍 Analyzing...
> Running step a65233ce-76ad-460c-9c29-fae858c5b7df. Step input: where is viet nam
Thought: The current language of the user is: English. I need to use external knowledge to answer this question, which is beyond the capabilities of the provided tools.
Answer: Vietnam is located in Southeast Asia, on the eastern coast of the Indochinese Peninsula.

📝 Final Answer:
Vietnam is located in Southeast Asia, on the eastern coast of the Indochinese Peninsula.

--------------------------------------------------

You: what is the sum of column likes

🔍 Analyzing...
> Running step 4a7345ee-cb77-4929-af57-a8aa32d90e0b. Step input: what is the sum of column likes
Thought: The current language of the user is: English. I need to use a tool to calculate the sum of the 'likes' column.
Action: sum_column
Action Input: {'column': 'likes'}
Observation: Error: [Errno 2] No such file or directory: '/content/combined_file.csv'


#Add Web Searching

In [5]:
%pip install tavily-python

In [6]:
from tavily import TavilyClient
import os
from google.colab import userdata
os.environ['TAVILY_API_KEY'] = userdata.get('tavily-key')

In [11]:
from tavily import AsyncTavilyClient

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    return str(await client.search(query))

In [10]:
tools.append(FunctionTool.from_defaults(fn=search_web, name="search_web"))

In [14]:
# ----- Create Agent with Reasoning Capabilities -----
class ReasoningAgent(ReActAgent):
    def _process_response(self, response):
        """Add reasoning visualization"""
        print(f"\n🤔 Agent Thoughts:")
        for step in response.response_gen:
            if step.action_reason:
                print(f"  - {step.action_reason}")
            if step.observation:
                print(f"  ✅ Observed: {step.observation}")
        return super()._process_response(response)

# Initialize agent
agent = ReasoningAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True,
    system_prompt="""You are an analytical assistant. Follow these steps:
    1. Understand the request in Vietnamese/English
    2. Check data availability
    3. Choose appropriate tool
    4. Verify results
    Respond in the user's language."""
)

# ----- Run Interaction Loop -----
print("📊 CSV Analysis Agent (Type 'exit' to quit)")
while True:
    query = input("\nYou: ")
    if query.lower() == 'exit':
        break

    print("\n🔍 Analyzing...")
    response = agent.chat(query)

    print(f"\n📝 Final Answer:")
    print(str(response))
    print("\n" + "-"*50)

print("Session ended. Goodbye!")

📊 CSV Analysis Agent (Type 'exit' to quit)

You: who is america president

🔍 Analyzing...
> Running step 3af0a3bb-7ee1-4a79-b4d5-b81b09849433. Step input: who is america president
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_web
Action Input: {'query': 'who is the president of the United States'}
Observation: {'query': 'who is the president of the United States', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Presidents, vice presidents, and first ladies - USAGov', 'url': 'https://www.usa.gov/presidents', 'content': 'An official website of the United States government A .gov website belongs to an official government organization in the United States. *   [A-Z index of U.S. government departments and agencies](https://www.usa.gov/agency-index) Learn about the duties of president, vice president, and first lady of the United States. President of the United States The president